In [1]:
# Imports
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import random
import time as t
import re
import sqlite3
from config import email
from config import password

In [2]:
# Create connection to the Fighter database establish the cursor look for existing table
con = sqlite3.connect('fighter_database.db')
cur = con.cursor()
cur.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='fighters' ''')

In [3]:
# Create table if it does not already exist
if cur.fetchone()[0]==1 : {
	print('Table exists.')
}
else:
    cur.execute('''CREATE TABLE fighters
               (name TEXT NOT NULL UNIQUE, kills INTEGER, deaths INTEGER, ratio REAL)''')

Table exists.


In [4]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\rab_b\.wdm\drivers\chromedriver\win32\103.0.5060.134]


In [5]:
#login to saltybet hit enter and wait 3 seconds for page to load
url = 'https://www.saltybet.com/authenticate?signin=1'
browser.visit(url)
#browser.find_by_id('email').click()
browser.find_by_id("email").fill(f'{email}')
browser.find_by_id("pword").fill(f'{password}\n')
# ActionChains(browser).move_to_element(text_input).send_keys_to_element(text_input, "mdicky93@gmail.com").perform()
t.sleep(3)

In [6]:
# function used to see if the bet was correct of not for statistics purposes
def cor_percent():
    # Correct_bet will go back to the main program for tracking stats
    global correct_bet
    error_catch = 0
    # gets banner information of who won. I know I did this in another function and this is likely extra work but i can clean later.
    winner_banner = browser.find_by_id('betstatus').value
    red_win = re.search('Payouts to Team Red', winner_banner)
    blue_win = re.search('Payouts to Team Blue', winner_banner)
    # looking to see who i bet on. I check it at this late in the game in case for some reason i changed who Richard bet on manually. though i realize that defeats the purpose of tracking Richard and not my own guesses.
    try:
        bet_red = browser.find_by_id("lastbet").find_by_css("span")[0].has_class('redtext')
        bet_blue = browser.find_by_id("lastbet").find_by_css("span")[0].has_class('bluetext')
        green_confirm = browser.find_by_id("lastbet").find_by_css("span")[1].has_class('greentext')
    # the except here makes it to where the next if statement does not fire and returns back giving correct_bet a value of 2 for the loop. This in turn shows as an error in the statistics.
    except:
        error_catch = 1
    if error_catch != 1:
        # want to make sure there is "color" text then "green" to confirm then just looking at all possible betting/winning senarios.
        if bet_red == True and green_confirm == True and red_win != None:
            correct_bet = 1
        elif bet_red == True and green_confirm == True and red_win == None:
            correct_bet = 0
        elif bet_blue == True and green_confirm == True and blue_win != None:
            correct_bet = 1
        elif bet_blue == True and green_confirm == True and blue_win == None:
            correct_bet = 0

In [7]:
# this function will update the fighters within the fighter_database.
def update_fighters(player1_text, player2_text):
    print('update start')
    # i put this sleep in because i think it will help with the webpage not fully loading the items i need to see in time. ocassionally i still get 2 values for correct_bet.
    t.sleep(0.1)
    winner_banner = browser.find_by_id('betstatus').value
    print(winner_banner)
    red_win = re.search('Payouts to Team Red', winner_banner)
    print(red_win)
    blue_win = re.search('Payouts to Team Blue', winner_banner)
    print(blue_win)
    print(f'red was {player1_text}  blue was {player2_text}')
    # re.search() will return None if it is not able to find the selected string within the winner_banner
    # for all the items below we will do the following
    # 1. check if the player is in the database
    # 2. if not INSERT them with values appropriate for the win/loss
    # 3. if the player is in the database we will UPDATE their entry with new win/loss and Ratio as appropriate.
    # 1st scenario is Red Wins
    if red_win != None:
        print('red won')
        cur.execute(' SELECT * from fighters WHERE name=?',(player1_text,))
        if cur.fetchone()==None:
            cur.execute("INSERT INTO fighters VALUES (?,?,?,?)", (player1_text,1,0,1.0))
            red_ratio = 1.0
        else:
            cur.execute(' SELECT kills, deaths from fighters WHERE name=?',(player1_text,))
            c_wins, c_loss = cur.fetchone()
            try:
                red_ratio = (c_wins + 1) / c_loss
            except ZeroDivisionError:
                red_ratio = (c_wins + 1)
            cur.execute('UPDATE fighters SET kills = ?, ratio = ? WHERE name = ?',(c_wins+1, red_ratio, player1_text))
        cur.execute(''' SELECT * from fighters WHERE name=?''',(player2_text,))    
        if cur.fetchone()==None:   
            cur.execute("INSERT INTO fighters VALUES (?,?,?,?)", (player2_text,0,1,0.0))
            blue_ratio = 0.0
        else:
            cur.execute(' SELECT kills, deaths from fighters WHERE name=?',(player2_text,))
            c_wins, c_loss = cur.fetchone()
            blue_ratio = c_wins / (c_loss + 1)
            cur.execute('UPDATE fighters SET deaths = ? , ratio = ? WHERE name = ?',(c_loss+1, blue_ratio, player2_text))
    # 2nd is Blue Wins
    elif blue_win != None:
        print('blue won')
        cur.execute(' SELECT * from fighters WHERE name=?',(player2_text,))
        if cur.fetchone()==None:
            cur.execute("INSERT INTO fighters VALUES (?,?,?,?)",(player2_text,1,0,1.0))
            blue_ratio = 1.0
        else:
            cur.execute(' SELECT kills, deaths from fighters WHERE name=?',(player2_text,))
            c_wins, c_loss = cur.fetchone()
            try:
                blue_ratio = (c_wins + 1) / c_loss
            except ZeroDivisionError:
                blue_ratio = (c_wins + 1)
            cur.execute('UPDATE fighters SET kills = ?, ratio = ? WHERE name = ?',(c_wins+1, blue_ratio, player2_text))
        cur.execute(''' SELECT * from fighters WHERE name=?''',(player1_text,))    
        if cur.fetchone()==None:   
            cur.execute("INSERT INTO fighters VALUES (?,?,?,?)",(player1_text,0,1,0.0))
        else:
            cur.execute(' SELECT kills, deaths from fighters WHERE name=?',(player1_text,))
            c_wins, c_loss = cur.fetchone()
            red_ratio = c_wins / (c_loss + 1)
            cur.execute('UPDATE fighters SET deaths = ? , ratio = ? WHERE name = ?',(c_loss+1, red_ratio, player1_text))
    # commit changes to the database.
    con.commit()
    print('fighters updated')
    # more error catching in that if the program for some reason sees that no player won it will return without updating the statistics. causeing a 2 value for correct_bet.
    if red_win == None and blue_win == None:
        return
    cor_percent()

In [8]:
# This is the loop that holds the program up until updating fighters and the next betting oppurtunity opens.
def bet_loop(player1_text,player2_text, exhib_marker):
    print('loop start')
    #check to make sure the bet was accepted before moving on
    # betconfirm is the little green check mark that shows up when you have confirmed your bet. it disappears at the start of matches.
    betconfirm = browser.is_element_present_by_id('betconfirm')
    if betconfirm == True:
        #don't want to constantly keep clicking random until match starts. bet confirm appears once you bet and disappears upon match start.
        i = 0
        while betconfirm == True:
            if i == 0:
                print('bet confirmed')
            betconfirm = browser.is_element_present_by_id('betconfirm')
            t.sleep(5)
            i += 1
        i = 0
        # betstatus is the string below the twitch window that show betting status (duh)
        while browser.find_by_id('betstatus').value == 'Bets are OPEN!' or browser.find_by_id('betstatus').value == 'Bets are locked until the next match.'\
        or browser.find_by_id('betstatus').value == "" or browser.find_by_id('betstatus').value == None:
            # this sleep is short because the window to see the winner of the fight is short.
            t.sleep(0.20)
            if i == 0:
                print('in a match')
            i += 1
        print('fighters to be updated')
        # exhibition marker comes from the main program line, we do not want to update stats if it is an exhibition match as we guess on those randomly.
        if exhib_marker == 1:
            return
        else:
            update_fighters(player1_text,player2_text)
    else:
        # if we don't get a bet confirmed we will wait 2 seconds and try again. Sometimes the website doesn't confirm the bet. nothing wrong with my program just a problem with the site.
        # also i need to look back on this and see how this will effect stats....
        print('bet not confirmed')
        t.sleep(2)
    print('loop complete')

In [9]:
# this is the function that decides who to bet on. There are 3 situations.
# the reason behind the mo_money variable was orignally this code was set up to click only the buttons for betting.
# however as Richard began to make more money 10% (the minimum button push) became a huge risk on the money Richard had.
# so instead of rewrite the code to incorperate betting a precentage i just added that functionality to the existing code.
# maybe one day i will go back and clean it up. sigh.
def bet_who(bet_amount, mo_money = 0):
    print('begin selection')
    player1 = browser.find_by_id('player1')
    player1_text = browser.find_by_id('player1').value
    # print(player1_text)
    player2 = browser.find_by_id('player2')
    player2_text = browser.find_by_id('player2').value
    balance = int(browser.find_by_id('balance').value.replace(',',""))
    # print(player2_text)
    cur.execute(' SELECT * from fighters WHERE name=?',(player1_text,))
    if cur.fetchone()==None:
        print('fighter 1 not found')
        fighter1_present=0
    else:
        print('fighter 1 found')
        fighter1_present=1
    cur.execute(' SELECT * from fighters WHERE name=?',(player2_text,))
    if cur.fetchone()==None:
        print('fighter 2 not found')
        fighter2_present=0
    else:
        print('fighter 2 found')
        fighter2_present=1
    # 1. Both fighters are not in the database.
    if fighter1_present == 0 and fighter2_present == 0:
        # random fight.
        try:
            #had to use a try except here as clicking when nothing is there causes errors. I'm sure this can be written better. this finds and clicks all in button
            if mo_money == 0:
                allin=browser.find_by_id(bet_amount)
                allin.click()
                t.sleep(0.2)
            else:
                browser.find_by_id('wager').fill(str(round(balance * mo_money)))
            #random click of red or blue.
            coinflip=random.choice([0,1])
            if coinflip == 1:
                player2.click()
                print('bet on blue')
                t.sleep(0.2)
            else:
                player1.click()
                print('bet on red')
                t.sleep(0.2)
            print('go to loop')
            exhib_marker = 0
            bet_loop(player1_text,player2_text, exhib_marker)
        except:
            t.sleep(1)
            # probably need to define a function for random then put here to retry random() after sleep
    # 2. Only 1 fighter is found
    elif (fighter1_present == 1 and fighter2_present == 0) or (fighter1_present == 0 and fighter2_present == 1):
        try:
            if mo_money == 0:
                allin=browser.find_by_id(bet_amount)
                allin.click()
                t.sleep(0.2)
            else:
                browser.find_by_id('wager').fill(str(round(balance * mo_money)))
            cur.execute(' SELECT * from fighters WHERE name=?',(player1_text,))
            if cur.fetchone()==None:
                cur.execute(' SELECT ratio from fighters WHERE name=?',(player2_text,))
                ratio = cur.fetchone()[0]
                print(f'player 2 ratio is {ratio}')
                if ratio < 1.0:
                    player1.click()
                    print('bet on red')
                    t.sleep(0.2)
                else:
                    player2.click()
                    print('bet on blue')
                    t.sleep(0.2)
            else:
                cur.execute(' SELECT ratio from fighters WHERE name=?',(player1_text,))
                ratio = cur.fetchone()[0]
                print(f'player 1 ratio is {ratio}')
                if ratio < 1.0:
                    player2.click()
                    print('bet on blue')
                    t.sleep(0.2)
                else:
                    player1.click()
                    print('bet on red')
                    t.sleep(0.2)
            print('go to loop')
            exhib_marker = 0
            bet_loop(player1_text,player2_text, exhib_marker)
        except:
            t.sleep(1)
        # bet on fighter with stats if ratio >0.5 else bet on unknown
    # 3. both fighters are found in the database.
    elif fighter1_present == 1 and fighter2_present == 1:
        # bet on fighter with higher win loss ratio
        try:
            if mo_money == 0:
                allin=browser.find_by_id(bet_amount)
                allin.click()
                t.sleep(0.2)
            else:
                browser.find_by_id('wager').fill(str(round(balance * mo_money)))
            cur.execute(''' SELECT ratio from fighters WHERE name=?''',(player1_text,)) 
            red_ratio = cur.fetchone()[0]
            print(f'red ratio is {red_ratio}')
            cur.execute(''' SELECT ratio from fighters WHERE name=?''',(player2_text,))
            blue_ratio = cur.fetchone()[0]
            print(f'blue ratio is {blue_ratio}')
            if red_ratio >= blue_ratio:
                player1.click()
                print('bet on red')
            else:
                player2.click()
                print('bet on blue')
            print('go to loop')
            exhib_marker = 0
            # the exhib_marker is what dictates wether the players are updated in the database or not at the end of the bet_loop function.
            bet_loop(player1_text,player2_text, exhib_marker)
        except:
            t.sleep(1)
    print('finish selection')

In [10]:
# there is a seperate version of betting for exhibitions because exhibitions are chaotic and unpredictable. but I still like betting so
# I made this alternate betting where the results are not logged in the database nor stats but bets are randomly made.
def exhib_bet():
    player1 = browser.find_by_id('player1')
    player1_text = browser.find_by_id('player1').value
    player2 = browser.find_by_id('player2')
    player2_text = browser.find_by_id('player2').value
    browser.find_by_id('wager').fill('1000')
    coinflip=random.choice([0,1])
    if coinflip == 1:
        player2.click()
        print('bet on blue')
        t.sleep(0.2)
    else:
        player1.click()
        print('bet on red')
        t.sleep(0.2)
    print('go to loop')
    t.sleep(0.2)
    # the exhib_marker is what dictates wether the players are updated in the database or not at the end of the bet_loop function.
    exhib_marker = 1
    bet_loop(player1_text,player2_text, exhib_marker)
        
   

In [11]:
# This function just prints stats on Richard.
def print_stats()
    print(times_correct)
    print(f'total number of matches: {u_input}')
    print(f'number of correct guesses: {times_correct.count(1)}')
    print(f'number of incorrect guesses: {times_correct.count(0)}')
    print(f'number of errors: {times_correct.count(2)}')
    try:
        print(f'percentage of right guesses: {round((times_correct.count(1)/(times_correct.count(1)+times_correct.count(0)))*100, 2)}%')        
    except ZeroDivisionError:
        print('no regular matches run yet')

In [12]:
u_input = int(input('How many matches would you like to run?'))
# intially sleep until there is the ability for Richard to bet.
while browser.find_by_id('betstatus').value != 'Bets are OPEN!':
    t.sleep(5)
times_correct = []
random.seed()
for i in range(u_input):
    # making sure that bets are open before getting into program at the begining of each loop.
    while browser.find_by_id('betstatus').value != 'Bets are OPEN!':
        t.sleep(1)
    # remove stats is set to 1 for exhibition matches so they are not included in the stats
    remove_stats = 0
    # this is a bug catcher for updating fighters/updating stats. this should be set to 1 or 0 by the cor_percent function.
    correct_bet = 2
    footer_text = browser.evaluate_script("document.getElementById('footer-alert').textContent")
    exhib_time = re.search('exhibition', footer_text, re.I)
    #used is_element_not instead of is_element_present as it runs almost 2 seconds faster, don't know why (edit: in documentation it says that a wait is
    # automatically applied to the is_element_present so that is_element_not will always run faster).
    t_time = browser.is_element_not_present_by_css('span[class="dollar purpletext"][id="balance"]')
    #if the purple text of balance is present we enter tournament betting
    if t_time == False and int(browser.find_by_id('balance').value.replace(',',"")) <=4000:
        print(f'{i} tournament all in')
        print(browser.find_by_id('balance').value)
        bet_who('interval10')
    elif t_time == False and int(browser.find_by_id('balance').value.replace(',',"")) <=7000:
        print(f'{i} tournament 50 percent')
        print(browser.find_by_id('balance').value)
        bet_who('interval5')
    elif t_time == False and int(browser.find_by_id('balance').value.replace(',',"")) > 7000:
        print(f'{i} tournament 10 percent')
        print(browser.find_by_id('balance').value)
        bet_who('interval1')
    # we put exhibition after tounament in this if-then list because the last round of tournament has the word exhibition and we don't want to enter
    # exhibtion betting to early. This was a bug that was caught.
    elif exhib_time != None and int(browser.find_by_id('balance').value.replace(',',"")) >=50000:
        print(f'{i} exhhibition bet')
        print(browser.find_by_id('balance').value)
        remove_stats = 1
        exhib_bet()
    elif exhib_time != None and int(browser.find_by_id('balance').value.replace(',',"")) <50000:
        y=0
        while exhib_time != None:
            if y== 0:
                print("exhibition not enough money")
                print(browser.find_by_id('balance').value)
                y += 1
            footer_text = browser.evaluate_script("document.getElementById('footer-alert').textContent")
            exhib_time = re.search('exhibition', footer_text, re.I)
            t.sleep(20)   
    # past this point we enter regular matchmaking betting. again, originally i had this set up to click the buttons for betting.
    # it wasn't till later that the betting as a percentage of balance was added. hence the weird setup. 
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 500:
        print(f'{i} regular all in')
        print(browser.find_by_id('balance').value)
        bet_who('interval10')
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 1000 :
        print(f'{i} regular half bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval5')
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 2000 :
        print(f'{i} regular 30 percent bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval3')
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 3000 :
        print(f'{i} regular 20 percent bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval2')
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 250000 :
        print(f'{i} regular 10 percent bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval1')
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 500000 :
        print(f'{i} regular 5% bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval1', 0.05)
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 2000000 :
        print(f'{i} regular 2.5% bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval1', 0.025)
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 3500000 :
        print(f'{i} regular 1.25% bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval1', 0.0125)
    elif int(browser.find_by_css('span[class="dollar"][id="balance"]').value.replace(',',"")) <= 5000000 :
        print(f'{i} regular 1.00% bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval1', 0.01)
    else:
        print(f'{i} regular 0.8% bet')
        print(browser.find_by_id('balance').value)
        bet_who('interval1', 0.008)
    if remove_stats == 0:
        times_correct.append(correct_bet)
    # print stats every 10 matches.
    if (i + 1) % 10 == 0:
        print_stats()
# can't forget to close browser and database conection after program has finished.
browser.quit()
con.close()

0 regular 0.8% bet
7,135,753
begin selection
fighter 1 not found
fighter 2 not found
bet on blue
go to loop
loop start
bet confirmed
in a match
fighters to be updated
update start
The Colonel wins! Payouts to Team Red.
<re.Match object; span=(18, 37), match='Payouts to Team Red'>
None
red was The Colonel  blue was Emeralda kasim
red won
fighters updated


c:\Users\rab_b\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:502: UserWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead")


loop complete
finish selection
1 regular 0.8% bet
7,078,667
begin selection
fighter 1 not found
fighter 2 found
player 2 ratio is 1.0
bet on blue
go to loop
loop start
bet confirmed
in a match
fighters to be updated
update start
Freddy krueger por m.l.c wins! Payouts to Team Blue.
None
<re.Match object; span=(31, 51), match='Payouts to Team Blue'>
red was Dr. ivo robotnik  blue was Freddy krueger por m.l.c
blue won
fighters updated
loop complete
finish selection
2 regular 0.8% bet
7,141,893
begin selection
fighter 1 found
fighter 2 found
red ratio is 2.0
blue ratio is 1.0
bet on red
go to loop
loop start
bet confirmed
in a match
fighters to be updated
update start
Do you believe it? wins! Payouts to Team Blue.
None
<re.Match object; span=(25, 45), match='Payouts to Team Blue'>
red was Cute syakoya  blue was Do you believe it?
blue won
fighters updated
loop complete
finish selection
3 regular 0.8% bet
7,084,758
begin selection
fighter 1 not found
fighter 2 not found
bet on red
go to loo